In [ ]:
#|default_exp app

In [ ]:
from duckduckgo_search import duckduckgo_search
from fastcore.all import *

def search_images(term, max_images=30):
    print(f"Searching for '{term}'")
    searcher = duckduckgo_search.DDGS()
    
    return L(searcher.images(term, size="Medium", max_results=max_images)).itemgot('image')

In [ ]:
#NB: `search_images` depends on duckduckgo.com, which doesn't always return correct responses.
#    If you get a JSON error, just try running it again (it may take a couple of tries).
urls = search_images('bird photos', max_images=1)
urls[0]

In [ ]:
from fastdownload import download_url
dest = 'bird.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
download_url(search_images('forest photos', max_images=1)[0], 'forest.jpg', show_progress=False)
Image.open('forest.jpg').to_thumb(256,256)

In [ ]:
searches = ['bird', 'forest']
path = Path('./data')
test_set = Path('./test')
from time import sleep
import os
import shutil

if path.exists():
    shutil.rmtree(path)


for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'{o}'))
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
if test_set.exists():
    shutil.rmtree(test_set)

# move some to the test set
for o in searches:
    imgs = get_image_files(path/o)
    test_no = round(len(imgs) * 0.2)
    testimgs = imgs[:test_no]
    for img in testimgs:
        dest = Path('./test')/img
        dest.mkdir(exist_ok=True, parents=True)
        shutil.move(img, dest)

In [ ]:
from duckduckgo_search import duckduckgo_search
from fastcore.all import *
from fastai.vision.all import *

path = Path('./data')
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(10)

In [ ]:
test_set = Path('./test')
files = get_image_files(test_set)

for imgpath in get_image_files(test_set):
    img = PILImage.create(imgpath)
    category,_,probs = learn.predict(img)
    
    im = Image.open(imgpath)
    display( im.to_thumb(256,256))
    print(f"This is {category}.")
    print(f"Probability it's a bird {probs[0]:.4f}")
    print(f"Probability it's a forest {probs[1]:.4f}")

In [8]:
# learn.save('myModel', with_opt=False)
learn.export('models/myModel.pkl')

In [13]:
#| export
from fastai.vision.all import load_learner
import gradio as gr

learn = load_learner('models/myModel.pkl')

def predict(img):
    category,_,probs = learn.predict(img)
    return category

demo = gr.Interface(
    fn=predict,
    inputs=["image"],
    outputs=["text"],
)

demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [15]:
import nbdev; nbdev.nbdev_export()

InterpolationMissingOptionError: Bad value substitution: option 'lib_name' in section 'DEFAULT' contains an interpolation key 'repo' which is not a valid option name. Raw value: '%(repo)s'